In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [11]:
spark = SparkSession.builder.appName("twitter_transformation").getOrCreate()

In [12]:
df_tweet = spark.read.json("../../datalake/silver/twitter_datascience/tweet")

In [13]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [14]:
tweet_conversas = df_tweet.alias("tweet").groupBy(f.to_date("created_at").alias("created_date")).agg(
    f.countDistinct("author_id").alias("n_tweets"),
    f.sum("like_count").alias("n_like"),
    f.sum("quote_count").alias("n_quote"),
    f.sum("reply_count").alias("n_replay"),
    f.sum("retweet_count").alias("n_retweet")
)